In [3]:
# 填充
# 给定(32x32)输入图像
# 应用5x5大小的卷积核
#   - 第1层得到输出大小28x28
#   - 第7层得到输出大小4x4
# 更大的卷积核可以更快地减小输出大小
#   - 形状从n_h x n_w 减少到
#   - (n_h - k_h +1) x (n_w - k_w +1)

# 填充 (padding)
# 在输入周围添加额外的行/列
# 填充(p_h)行和(p_w)列, 输出形状为 (n_h - k_h + p_h -1) x (n_w - k_w + p_w -1)
# 通常取 p_h = k_h -1, p_w = k_w -1
# 因为这样, 将p_h和p_w带入后, 可以发现 输出形状变为 (n_h) x (n_w), 也就是"不会改变输出的形状"
#   - 当 k_h 为奇数: 在上下两侧填充 p_h/2
#   - 当 k_h 为偶数, 在上侧填充ceiling(p_h/2), floor(p_h/2) --> 我们基本不会用偶数卷积核

# 步幅 (stride)
# 填充减小的输出大小与线性层相关
#   - 给定输入大小224x224, 在使用5x5卷积核的情况下, 需要44层将输出降低为4x4
#   - 需要大量计算才能得到较小的输出
# 步幅是指行/列的滑动步长
# 例: 高度3 宽度2 的步幅
# 当没得跳一个完全的步幅时, 停止

# 步幅
# 给定高度s_h和宽度s_w的步幅, 输出的形状是 
#    - floor((n_h - k_h + p_h + s_h)/s_h) x floor((n_w - k_w + p_w + s_w)/s_w)
#    - 为什么这里是 + s_h/w? 可以理解为我们先减去 k占据的部分, 如果末尾部分不满足stride, 就用floor将他取0
# 如果p_h = k_h -1, p_w = k_w -1
#    - floor((n_h + s_h -1)/s_h) x floor((n_w + s_w -1)/s_w)
# 如果输入的高度, 宽度都可以被步幅整除
#    - （n_h/s_h) x (n_w/s_w)

# 总结
# 填充和步幅是卷积层的超参数
# 填充在输入周围添加额外的行/列, 来控制输出的形状的减少量
# 步幅是每次滑动核窗口时的行/列的步长, 可以成倍的减少输出形状

In [4]:
# 填充和步幅
# 在所有侧边填充1个像素
import torch
from torch import nn

def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape) # (1,1)是通道数和批量大小数; X.shape是维度
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:]) # 输出是个4维的矩阵, 将前2维拿掉

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1) #输入的通道数和批量大小都是1, Kernel大小为3x3, padding为1
X = torch.rand(size= (8,8)) # 生成一个随机的8x8的矩阵
comp_conv2d(conv2d, X).shape # 输出是一个8x8, 因为 (8+2(padding *2) +1 -3) = 8

torch.Size([8, 8])

In [6]:
# 填充不同的高度和宽度
conv2d = nn.Conv2d(1, 1, kernel_size=(5,3), padding=(2,1)) # 上下填充的行数是2, 左右填充的列数是1
# height = 8 - 5 + 2*2 + 1 = 8
# width = 8 - 3 + 1*2 +1 = 8
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [8]:
# 将高度和宽度的步幅设置为2
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
# (8-3+1*2+1)/2 = 4 
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [9]:
# 一个稍微复杂的例子
conv2d = nn.Conv2d(1, 1, kernel_size=(3,5), padding=(0,1), stride=(3,4))
# floor((8-3+0*2)/4+1) = 2
# floor((8-5+1*2)/3+1) = 2
# 注意这里除数是要换
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])